## Part 1: Setup & Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Paths
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'

print(f"✓ Project root: {PROJECT_ROOT}")
print(f"✓ Raw data: {RAW_DIR}")
print(f"✓ Processed data: {PROCESSED_DIR}")

✓ Project root: /home/sakana/Code/DS-RS
✓ Raw data: /home/sakana/Code/DS-RS/data/raw
✓ Processed data: /home/sakana/Code/DS-RS/data/processed


## Part 2: Helper Functions

### 2.1 Parse Location

In [2]:
def parse_location(loc_str: str) -> Dict[str, str]:
    """
    Parse location string into city, state, country.
    
    Examples:
    - "San Francisco, CA, United States" → {'city': 'San Francisco', 'state': 'CA', 'country': 'United States'}
    - "New York, NY" → {'city': 'New York', 'state': 'NY', 'country': 'United States'}
    - "Remote" → {'city': 'Remote', 'state': '', 'country': ''}
    - "United States" → {'city': '', 'state': '', 'country': 'United States'}
    """
    if pd.isna(loc_str) or not isinstance(loc_str, str) or loc_str.strip() == '':
        return {'city': '', 'state': '', 'country': ''}
    
    location = loc_str.strip()
    
    # Special cases
    if location.lower() == 'remote':
        return {'city': 'Remote', 'state': '', 'country': ''}
    
    if location == 'United States':
        return {'city': '', 'state': '', 'country': 'United States'}
    
    # Split by comma
    parts = [p.strip() for p in location.split(',')]
    
    if len(parts) == 1:
        # Only one part - could be city or country
        return {'city': parts[0], 'state': '', 'country': ''}
    
    elif len(parts) == 2:
        # Two parts - city, state OR city, country
        city, second = parts
        # If second part is 2 uppercase letters, likely US state
        if len(second) == 2 and second.isupper():
            return {'city': city, 'state': second, 'country': 'United States'}
        else:
            return {'city': city, 'state': '', 'country': second}
    
    else:
        # Three or more parts - city, state, country
        return {
            'city': parts[0],
            'state': parts[1],
            'country': parts[-1]
        }

# Test
test_cases = [
    "San Francisco, CA, United States",
    "New York, NY",
    "Remote",
    "United States",
    "London, United Kingdom",
    None,
    ""
]

print("Testing parse_location():")
for test in test_cases:
    result = parse_location(test)
    print(f"  {test!r:40s} → {result}")

Testing parse_location():
  'San Francisco, CA, United States'       → {'city': 'San Francisco', 'state': 'CA', 'country': 'United States'}
  'New York, NY'                           → {'city': 'New York', 'state': 'NY', 'country': 'United States'}
  'Remote'                                 → {'city': 'Remote', 'state': '', 'country': ''}
  'United States'                          → {'city': '', 'state': '', 'country': 'United States'}
  'London, United Kingdom'                 → {'city': 'London', 'state': '', 'country': 'United Kingdom'}
  None                                     → {'city': '', 'state': '', 'country': ''}
  ''                                       → {'city': '', 'state': '', 'country': ''}


### 2.2 Normalize Salary to Yearly

In [3]:
def normalize_salary_to_yearly(row: pd.Series) -> Optional[float]:
    """
    Convert salary to yearly amount.
    
    Args:
        row: pandas Series with columns: min_salary, max_salary, pay_period
    
    Returns:
        Yearly salary (median of min and max), or None if missing
    
    Multipliers:
    - YEARLY: 1
    - MONTHLY: 12
    - BIWEEKLY: 26
    - WEEKLY: 52
    - HOURLY: 2080 (40 hours/week × 52 weeks)
    """
    # Check if salary data exists
    if pd.isna(row.get('min_salary')) or pd.isna(row.get('max_salary')):
        return None
    
    # Calculate median
    try:
        min_sal = float(row['min_salary'])
        max_sal = float(row['max_salary'])
        median = (min_sal + max_sal) / 2
    except (ValueError, TypeError):
        return None
    
    # Get pay period
    period = str(row.get('pay_period', '')).upper()
    
    # Conversion multipliers
    multipliers = {
        'YEARLY': 1,
        'MONTHLY': 12,
        'BIWEEKLY': 26,
        'WEEKLY': 52,
        'HOURLY': 2080,  # 40h/week × 52 weeks
    }
    
    multiplier = multipliers.get(period, 1)
    return median * multiplier

# Test
test_data = pd.DataFrame([
    {'min_salary': 80000, 'max_salary': 120000, 'pay_period': 'YEARLY'},
    {'min_salary': 25, 'max_salary': 35, 'pay_period': 'HOURLY'},
    {'min_salary': 5000, 'max_salary': 7000, 'pay_period': 'MONTHLY'},
    {'min_salary': None, 'max_salary': 100000, 'pay_period': 'YEARLY'},
])

print("Testing normalize_salary_to_yearly():")
for idx, row in test_data.iterrows():
    result = normalize_salary_to_yearly(row)
    print(f"  {row['min_salary']}-{row['max_salary']} {row['pay_period']:10s} → ${result:,.0f}" if result else f"  {row['min_salary']}-{row['max_salary']} {row['pay_period']:10s} → None")

Testing normalize_salary_to_yearly():
  80000.0-120000 YEARLY     → $100,000
  25.0-35 HOURLY     → $62,400
  5000.0-7000 MONTHLY    → $72,000
  nan-100000 YEARLY     → None


## Part 3: Data Loading Functions

### 3.1 Load Jobs (Normalized - NO Aggregation)

In [4]:
def load_jobs_normalized(sample: Optional[int] = None) -> pd.DataFrame:
    """
    Load jobs table without aggregation.
    
    Args:
        sample: If not None, only load first N rows (for testing)
    
    Returns:
        DataFrame with columns:
        - job_id, title, description, company_id, company_name
        - location, city, state, country
        - work_type, experience_level, remote_allowed
        - min_salary, max_salary, pay_period, normalized_salary_yearly
        - views, applies, listed_time, closed_time
    """
    print("\n" + "="*60)
    print("LOADING JOBS (Normalized)")
    print("="*60)
    
    # 1. Load raw postings
    print("Step 1: Reading postings.csv...")
    postings_path = RAW_DIR / 'postings.csv'
    
    nrows = sample if sample else None
    postings = pd.read_csv(postings_path, nrows=nrows)
    print(f"  ✓ Loaded {len(postings):,} rows")
    
    # 2. Select columns
    print("\nStep 2: Selecting columns...")
    jobs = postings[[
        'job_id', 'title', 'description', 'company_id', 'company_name',
        'location', 'formatted_work_type', 'formatted_experience_level',
        'remote_allowed', 'min_salary', 'max_salary', 'pay_period',
        'views', 'applies', 'original_listed_time', 'closed_time'
    ]].copy()
    
    # 3. Clean
    print("\nStep 3: Cleaning data...")
    
    # Drop jobs without title or description
    before = len(jobs)
    jobs = jobs[jobs['title'].notna() & (jobs['title'].str.strip() != '')]
    jobs = jobs[jobs['description'].notna() & (jobs['description'].str.strip() != '')]
    print(f"  ✓ Dropped {before - len(jobs):,} rows with missing title/description")
    
    # Remove duplicates by job_id
    before = len(jobs)
    jobs = jobs.drop_duplicates(subset=['job_id'], keep='first')
    print(f"  ✓ Dropped {before - len(jobs):,} duplicate job_id rows")
    
    # 4. Parse location
    print("\nStep 4: Parsing location...")
    location_parsed = jobs['location'].fillna('').apply(parse_location)
    jobs['city'] = location_parsed.apply(lambda x: x['city'])
    jobs['state'] = location_parsed.apply(lambda x: x['state'])
    jobs['country'] = location_parsed.apply(lambda x: x['country'])
    print(f"  ✓ Parsed into city, state, country")
    
    # 5. Normalize salary
    print("\nStep 5: Normalizing salary to yearly...")
    jobs['normalized_salary_yearly'] = jobs.apply(normalize_salary_to_yearly, axis=1)
    salary_count = jobs['normalized_salary_yearly'].notna().sum()
    print(f"  ✓ {salary_count:,} jobs have salary data ({salary_count/len(jobs)*100:.1f}%)")
    
    # 6. Rename columns
    jobs = jobs.rename(columns={
        'formatted_work_type': 'work_type',
        'formatted_experience_level': 'experience_level',
        'original_listed_time': 'listed_time'
    })
    
    # 7. Convert dtypes
    print("\nStep 6: Converting data types...")
    jobs['job_id'] = pd.to_numeric(jobs['job_id'], errors='coerce').astype('Int64')
    jobs['company_id'] = pd.to_numeric(jobs['company_id'], errors='coerce').astype('Int64')
    jobs['remote_allowed'] = jobs['remote_allowed'].astype('boolean')
    jobs['listed_time'] = pd.to_datetime(jobs['listed_time'], errors='coerce')
    jobs['closed_time'] = pd.to_datetime(jobs['closed_time'], errors='coerce')
    print(f"  ✓ Converted dtypes")
    
    print(f"\n✓ Final shape: {jobs.shape}")
    print(f"✓ Memory usage: {jobs.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    return jobs

# Test with sample
print("Testing with sample (1000 rows)...")
jobs_sample = load_jobs_normalized(sample=1000)
print("\nSample data:")
jobs_sample[['job_id', 'title', 'city', 'state', 'country', 'work_type']].head()

Testing with sample (1000 rows)...

LOADING JOBS (Normalized)
Step 1: Reading postings.csv...
  ✓ Loaded 1,000 rows

Step 2: Selecting columns...

Step 3: Cleaning data...
  ✓ Dropped 0 rows with missing title/description
  ✓ Dropped 0 duplicate job_id rows

Step 4: Parsing location...
  ✓ Parsed into city, state, country

Step 5: Normalizing salary to yearly...
  ✓ 338 jobs have salary data (33.8%)

Step 6: Converting data types...
  ✓ Converted dtypes

✓ Final shape: (1000, 20)
✓ Memory usage: 5.4 MB

Sample data:


,job_id,title,city,state,country,work_type
0,921716,Marketing Coordinator,Princeton,NJ,United States,Full-time
1,1829192,Mental Health Therapist/Counselor,Fort Collins,CO,United States,Full-time
2,10998357,Assitant Restaurant Manager,Cincinnati,OH,United States,Full-time
3,23221523,Senior Elder Law / Trusts and Estates Associat...,New Hyde Park,NY,United States,Full-time
4,35982263,Service Technician,Burlington,IA,United States,Full-time


### 3.2 Load Job Skills (Many-to-Many)

In [5]:
def load_job_skills() -> pd.DataFrame:
    """
    Load job-skill relationships (NO aggregation).
    
    Returns:
        DataFrame with columns: job_id, skill_abr
    """
    print("\n" + "="*60)
    print("LOADING JOB_SKILLS (Many-to-Many)")
    print("="*60)
    
    path = RAW_DIR / 'jobs' / 'job_skills.csv'
    job_skills = pd.read_csv(path)
    
    # Keep only needed columns
    job_skills = job_skills[['job_id', 'skill_abr']].copy()
    
    # Convert dtypes
    job_skills['job_id'] = pd.to_numeric(job_skills['job_id'], errors='coerce').astype('Int64')
    
    # Drop rows with missing values
    before = len(job_skills)
    job_skills = job_skills.dropna()
    
    print(f"✓ Loaded {len(job_skills):,} job-skill relationships")
    print(f"✓ Dropped {before - len(job_skills):,} rows with missing values")
    print(f"✓ Memory usage: {job_skills.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    return job_skills

# Test
job_skills = load_job_skills()
print("\nSample data:")
print(job_skills.head(10))
print(f"\nUnique jobs: {job_skills['job_id'].nunique():,}")
print(f"Unique skills: {job_skills['skill_abr'].nunique():,}")


LOADING JOB_SKILLS (Many-to-Many)
✓ Loaded 213,768 job-skill relationships
✓ Dropped 0 rows with missing values
✓ Memory usage: 12.5 MB

Sample data:
       job_id skill_abr
0  3884428798      MRKT
1  3884428798        PR
2  3884428798       WRT
3  3887473071      SALE
4  3887465684       FIN
5  3887465684      SALE
6  3887467939      SALE
7  3887467939      ADVR
8  3887467939        BD
9  3887471331       ENG

Unique jobs: 126,807
Unique skills: 35


### 3.3 Load Skills Lookup Table

In [6]:
def load_skills() -> pd.DataFrame:
    """
    Load skills lookup table.
    
    Returns:
        DataFrame with columns: skill_abr, skill_name
    """
    print("\n" + "="*60)
    print("LOADING SKILLS (Lookup Table)")
    print("="*60)
    
    path = RAW_DIR / 'mappings' / 'skills.csv'
    skills = pd.read_csv(path)
    skills = skills[['skill_abr', 'skill_name']].copy()
    
    print(f"✓ Loaded {len(skills):,} skills")
    
    return skills

# Test
skills = load_skills()
print("\nAll skills:")
print(skills)


LOADING SKILLS (Lookup Table)
✓ Loaded 35 skills

All skills:
   skill_abr              skill_name
0        ART            Art/Creative
1       DSGN                  Design
2       ADVR             Advertising
3       PRDM      Product Management
4       DIST            Distribution
5        EDU               Education
6       TRNG                Training
7       PRJM      Project Management
8       CNSL              Consulting
9       PRCH              Purchasing
10      SUPL            Supply Chain
11      ANLS                 Analyst
12      HCPR    Health Care Provider
13      RSCH                Research
14       SCI                 Science
15      GENB        General Business
16      CUST        Customer Service
17      STRA       Strategy/Planning
18       FIN                 Finance
19      OTHR                   Other
20       LGL                   Legal
21       ENG             Engineering
22        QA       Quality Assurance
23        BD    Business Development
24        IT

### 3.4 Load Job Industries (Many-to-Many)

In [7]:
def load_job_industries() -> pd.DataFrame:
    """
    Load job-industry relationships (NO aggregation).
    
    Returns:
        DataFrame with columns: job_id, industry_id
    """
    print("\n" + "="*60)
    print("LOADING JOB_INDUSTRIES (Many-to-Many)")
    print("="*60)
    
    path = RAW_DIR / 'jobs' / 'job_industries.csv'
    job_industries = pd.read_csv(path)
    
    # Keep only needed columns
    job_industries = job_industries[['job_id', 'industry_id']].copy()
    
    # Convert dtypes
    job_industries['job_id'] = pd.to_numeric(job_industries['job_id'], errors='coerce').astype('Int64')
    job_industries['industry_id'] = pd.to_numeric(job_industries['industry_id'], errors='coerce').astype('Int64')
    
    # Drop rows with missing values
    before = len(job_industries)
    job_industries = job_industries.dropna()
    
    print(f"✓ Loaded {len(job_industries):,} job-industry relationships")
    print(f"✓ Dropped {before - len(job_industries):,} rows with missing values")
    print(f"✓ Memory usage: {job_industries.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    return job_industries

# Test
job_industries = load_job_industries()
print("\nSample data:")
print(job_industries.head(10))
print(f"\nUnique jobs: {job_industries['job_id'].nunique():,}")
print(f"Unique industries: {job_industries['industry_id'].nunique():,}")


LOADING JOB_INDUSTRIES (Many-to-Many)
✓ Loaded 164,808 job-industry relationships
✓ Dropped 0 rows with missing values
✓ Memory usage: 2.8 MB

Sample data:
       job_id  industry_id
0  3884428798           82
1  3887473071           48
2  3887465684           41
3  3887467939           82
4  3887467939           80
5  3887471331           57
6  3887471331          332
7  3887471331          383
8  3887471274           82
9  3887471274           80

Unique jobs: 127,125
Unique industries: 422


### 3.5 Load Industries Lookup Table

In [8]:
def load_industries() -> pd.DataFrame:
    """
    Load industries lookup table.
    
    Returns:
        DataFrame with columns: industry_id, industry_name
    """
    print("\n" + "="*60)
    print("LOADING INDUSTRIES (Lookup Table)")
    print("="*60)
    
    path = RAW_DIR / 'mappings' / 'industries.csv'
    industries = pd.read_csv(path)
    industries = industries[['industry_id', 'industry_name']].copy()
    
    print(f"✓ Loaded {len(industries):,} industries")
    
    return industries

# Test
industries = load_industries()
print("\nSample industries:")
print(industries.head(20))


LOADING INDUSTRIES (Lookup Table)
✓ Loaded 422 industries

Sample industries:
    industry_id                         industry_name
0             1       Defense and Space Manufacturing
1             3       Computer Hardware Manufacturing
2             4                  Software Development
3             5          Computer Networking Products
4             6  Technology, Information and Internet
5             7           Semiconductor Manufacturing
6             8                    Telecommunications
7             9                          Law Practice
8            10                        Legal Services
9            11      Business Consulting and Services
10           12                Biotechnology Research
11           13                     Medical Practices
12           14             Hospitals and Health Care
13           15          Pharmaceutical Manufacturing
14           16                   Veterinary Services
15           17       Medical Equipment Manufacturing
16 

## Part 4: Verification với Sample Data

### 4.1 Test JOIN logic

In [9]:
# Test với 1 job cụ thể
test_job_id = jobs_sample['job_id'].iloc[0]

print(f"Testing với job_id = {test_job_id}")
print("\n" + "="*60)

# Job info
job_info = jobs_sample[jobs_sample['job_id'] == test_job_id][['title', 'company_name', 'city', 'state']].iloc[0]
print(f"Job: {job_info['title']}")
print(f"Company: {job_info['company_name']}")
print(f"Location: {job_info['city']}, {job_info['state']}")

# Skills cho job này
job_skill_list = job_skills[job_skills['job_id'] == test_job_id]
print(f"\nSkills ({len(job_skill_list)} skills):")
for _, row in job_skill_list.iterrows():
    skill_name = skills[skills['skill_abr'] == row['skill_abr']]['skill_name'].values
    skill_name = skill_name[0] if len(skill_name) > 0 else 'Unknown'
    print(f"  - {row['skill_abr']}: {skill_name}")

# Industries cho job này
job_industry_list = job_industries[job_industries['job_id'] == test_job_id]
print(f"\nIndustries ({len(job_industry_list)} industries):")
for _, row in job_industry_list.iterrows():
    industry_name = industries[industries['industry_id'] == row['industry_id']]['industry_name'].values
    industry_name = industry_name[0] if len(industry_name) > 0 else 'Unknown'
    print(f"  - {row['industry_id']}: {industry_name}")

print("\n" + "="*60)
print("✓ JOIN logic hoạt động đúng!")
print("✓ Skills và Industries KHÔNG bị aggregate thành string")

Testing với job_id = 921716

Job: Marketing Coordinator
Company: Corcoran Sawyer Smith
Location: Princeton, NJ

Skills (2 skills):
  - MRKT: Marketing
  - SALE: Sales

Industries (1 industries):
  - 44: Real Estate

✓ JOIN logic hoạt động đúng!
✓ Skills và Industries KHÔNG bị aggregate thành string


### 4.2 Verify Data Quality

In [10]:
def verify_data_quality(jobs, job_skills, skills, job_industries, industries):
    """Verify data quality của normalized tables."""
    
    print("\n" + "="*60)
    print("DATA QUALITY VERIFICATION")
    print("="*60)
    
    # 1. Jobs table
    print("\n1. JOBS TABLE")
    print(f"   ✓ Rows: {len(jobs):,}")
    print(f"   ✓ Unique job_id: {jobs['job_id'].is_unique}")
    print(f"   ✓ No missing title: {jobs['title'].notna().all()}")
    print(f"   ✓ No missing description: {jobs['description'].notna().all()}")
    print(f"   ✓ Salary coverage: {jobs['normalized_salary_yearly'].notna().sum()/len(jobs)*100:.1f}%")
    
    # 2. Job Skills
    print("\n2. JOB_SKILLS TABLE")
    print(f"   ✓ Rows: {len(job_skills):,}")
    print(f"   ✓ Unique jobs: {job_skills['job_id'].nunique():,}")
    print(f"   ✓ Unique skills: {job_skills['skill_abr'].nunique():,}")
    print(f"   ✓ Avg skills per job: {len(job_skills)/job_skills['job_id'].nunique():.2f}")
    
    # 3. Skills lookup
    print("\n3. SKILLS LOOKUP")
    print(f"   ✓ Total skills: {len(skills):,}")
    
    # 4. Job Industries
    print("\n4. JOB_INDUSTRIES TABLE")
    print(f"   ✓ Rows: {len(job_industries):,}")
    print(f"   ✓ Unique jobs: {job_industries['job_id'].nunique():,}")
    print(f"   ✓ Unique industries: {job_industries['industry_id'].nunique():,}")
    print(f"   ✓ Avg industries per job: {len(job_industries)/job_industries['job_id'].nunique():.2f}")
    
    # 5. Industries lookup
    print("\n5. INDUSTRIES LOOKUP")
    print(f"   ✓ Total industries: {len(industries):,}")
    
    # 6. Foreign key integrity
    print("\n6. FOREIGN KEY INTEGRITY")
    
    # Check job_skills references valid skills
    invalid_skills = set(job_skills['skill_abr']) - set(skills['skill_abr'])
    print(f"   ✓ Invalid skills in job_skills: {len(invalid_skills)}")
    
    # Check job_industries references valid industries
    invalid_industries = set(job_industries['industry_id']) - set(industries['industry_id'])
    print(f"   ✓ Invalid industries in job_industries: {len(invalid_industries)}")
    
    print("\n" + "="*60)
    print("✓ ALL CHECKS PASSED!")
    print("="*60)

# Run verification với sample data
verify_data_quality(jobs_sample, job_skills, skills, job_industries, industries)


DATA QUALITY VERIFICATION

1. JOBS TABLE
   ✓ Rows: 1,000
   ✓ Unique job_id: True
   ✓ No missing title: True
   ✓ No missing description: True
   ✓ Salary coverage: 33.8%

2. JOB_SKILLS TABLE
   ✓ Rows: 213,768
   ✓ Unique jobs: 126,807
   ✓ Unique skills: 35
   ✓ Avg skills per job: 1.69

3. SKILLS LOOKUP
   ✓ Total skills: 35

4. JOB_INDUSTRIES TABLE
   ✓ Rows: 164,808
   ✓ Unique jobs: 127,125
   ✓ Unique industries: 422
   ✓ Avg industries per job: 1.30

5. INDUSTRIES LOOKUP
   ✓ Total industries: 422

6. FOREIGN KEY INTEGRITY
   ✓ Invalid skills in job_skills: 0
   ✓ Invalid industries in job_industries: 0

✓ ALL CHECKS PASSED!


## Part 5: Save Functions

### 5.1 Save Normalized Data

In [11]:
def save_normalized_data(jobs, job_skills, skills, job_industries, industries):
    """
    Save all normalized tables to data/processed/
    """
    print("\n" + "="*60)
    print("SAVING NORMALIZED DATA")
    print("="*60)
    
    # Create directory if not exists
    PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
    
    # Save as Parquet (compact, fast)
    files = {
        'jobs.parquet': jobs,
        'job_skills.parquet': job_skills,
        'skills.parquet': skills,
        'job_industries.parquet': job_industries,
        'industries.parquet': industries,
    }
    
    total_size = 0
    for filename, df in files.items():
        filepath = PROCESSED_DIR / filename
        df.to_parquet(filepath, index=False)
        
        # Get file size
        size_mb = filepath.stat().st_size / 1024**2
        total_size += size_mb
        
        print(f"  ✓ {filename:25s} {size_mb:6.1f} MB  ({len(df):,} rows)")
    
    print(f"\n  Total: {total_size:.1f} MB")
    print(f"\n✓ All files saved to: {PROCESSED_DIR}")
    print("="*60)

# Test save với sample data
print("Testing save function với sample data...")
save_normalized_data(jobs_sample, job_skills, skills, job_industries, industries)

Testing save function với sample data...

SAVING NORMALIZED DATA
  ✓ jobs.parquet                 1.6 MB  (1,000 rows)
  ✓ job_skills.parquet           1.1 MB  (213,768 rows)
  ✓ skills.parquet               0.0 MB  (35 rows)
  ✓ job_industries.parquet       1.0 MB  (164,808 rows)
  ✓ industries.parquet           0.0 MB  (422 rows)

  Total: 3.8 MB

✓ All files saved to: /home/sakana/Code/DS-RS/data/processed


## Part 6: Run Full Pipeline

### 6.1 Load Full Dataset

In [12]:
# WARNING: This will load full dataset (~123k jobs)
# Uncomment to run

print("⚠️  LOADING FULL DATASET - This will take 2-3 minutes...")
print("")

# Load all data
jobs_full = load_jobs_normalized(sample=None)  # Full dataset
job_skills_full = load_job_skills()
skills_full = load_skills()
job_industries_full = load_job_industries()
industries_full = load_industries()

print("\n✓ All data loaded successfully!")

⚠️  LOADING FULL DATASET - This will take 2-3 minutes...


LOADING JOBS (Normalized)
Step 1: Reading postings.csv...
  ✓ Loaded 123,849 rows

Step 2: Selecting columns...

Step 3: Cleaning data...
  ✓ Dropped 7 rows with missing title/description
  ✓ Dropped 0 duplicate job_id rows

Step 4: Parsing location...
  ✓ Parsed into city, state, country

Step 5: Normalizing salary to yearly...
  ✓ 29,792 jobs have salary data (24.1%)

Step 6: Converting data types...
  ✓ Converted dtypes

✓ Final shape: (123842, 20)
✓ Memory usage: 879.3 MB

LOADING JOB_SKILLS (Many-to-Many)
✓ Loaded 213,768 job-skill relationships
✓ Dropped 0 rows with missing values
✓ Memory usage: 12.5 MB

LOADING SKILLS (Lookup Table)
✓ Loaded 35 skills

LOADING JOB_INDUSTRIES (Many-to-Many)
✓ Loaded 164,808 job-industry relationships
✓ Dropped 0 rows with missing values
✓ Memory usage: 2.8 MB

LOADING INDUSTRIES (Lookup Table)
✓ Loaded 422 industries

✓ All data loaded successfully!


### 6.2 Verify Full Dataset

In [13]:
# Verify full dataset
verify_data_quality(
    jobs_full, 
    job_skills_full, 
    skills_full, 
    job_industries_full, 
    industries_full
)


DATA QUALITY VERIFICATION

1. JOBS TABLE
   ✓ Rows: 123,842
   ✓ Unique job_id: True
   ✓ No missing title: True
   ✓ No missing description: True
   ✓ Salary coverage: 24.1%

2. JOB_SKILLS TABLE
   ✓ Rows: 213,768
   ✓ Unique jobs: 126,807
   ✓ Unique skills: 35
   ✓ Avg skills per job: 1.69

3. SKILLS LOOKUP
   ✓ Total skills: 35

4. JOB_INDUSTRIES TABLE
   ✓ Rows: 164,808
   ✓ Unique jobs: 127,125
   ✓ Unique industries: 422
   ✓ Avg industries per job: 1.30

5. INDUSTRIES LOOKUP
   ✓ Total industries: 422

6. FOREIGN KEY INTEGRITY
   ✓ Invalid skills in job_skills: 0
   ✓ Invalid industries in job_industries: 0

✓ ALL CHECKS PASSED!


### 6.3 Save Full Dataset

In [14]:
# Save full dataset
save_normalized_data(
    jobs_full, 
    job_skills_full, 
    skills_full, 
    job_industries_full, 
    industries_full
)

print("\n" + "="*60)
print("🎉 DAY 1 COMPLETE!")
print("="*60)
print("\n✅ Normalized data pipeline ready")
print("✅ NO aggregation - skills/industries kept separate")
print("✅ All data saved to data/processed/")
print("\nNext: Day 2 - BM25 Search Implementation")


SAVING NORMALIZED DATA
  ✓ jobs.parquet               224.7 MB  (123,842 rows)
  ✓ job_skills.parquet           1.1 MB  (213,768 rows)
  ✓ skills.parquet               0.0 MB  (35 rows)
  ✓ job_industries.parquet       1.0 MB  (164,808 rows)
  ✓ industries.parquet           0.0 MB  (422 rows)

  Total: 226.9 MB

✓ All files saved to: /home/sakana/Code/DS-RS/data/processed

🎉 DAY 1 COMPLETE!

✅ Normalized data pipeline ready
✅ NO aggregation - skills/industries kept separate
✅ All data saved to data/processed/

Next: Day 2 - BM25 Search Implementation


## Part 7: Final Summary & Storage Comparison

In [15]:
import os

def get_directory_size(path):
    """Calculate total size of all files in directory."""
    total = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
        elif entry.is_dir():
            total += get_directory_size(entry.path)
    return total

# Compare storage
print("\n" + "="*60)
print("STORAGE COMPARISON")
print("="*60)

# Old (aggregated)
old_file = PROCESSED_DIR / 'clean_jobs.parquet'
if old_file.exists():
    old_size = old_file.stat().st_size / 1024**2
    print(f"\n❌ OLD (Aggregated):")
    print(f"   clean_jobs.parquet: {old_size:.1f} MB")

# New (normalized)
new_files = ['jobs.parquet', 'job_skills.parquet', 'skills.parquet', 
             'job_industries.parquet', 'industries.parquet']
new_total = 0

print(f"\n✅ NEW (Normalized):")
for filename in new_files:
    filepath = PROCESSED_DIR / filename
    if filepath.exists():
        size = filepath.stat().st_size / 1024**2
        new_total += size
        print(f"   {filename:25s} {size:6.1f} MB")

print(f"\n   TOTAL: {new_total:.1f} MB")

if old_file.exists():
    savings = (old_size - new_total) / old_size * 100
    print(f"\n💾 SAVINGS: {old_size - new_total:.1f} MB ({savings:.1f}%)")

print("="*60)


STORAGE COMPARISON

❌ OLD (Aggregated):
   clean_jobs.parquet: 674.4 MB

✅ NEW (Normalized):
   jobs.parquet               224.7 MB
   job_skills.parquet           1.1 MB
   skills.parquet               0.0 MB
   job_industries.parquet       1.0 MB
   industries.parquet           0.0 MB

   TOTAL: 226.9 MB

💾 SAVINGS: 447.5 MB (66.4%)
